# Getting Started with Strands Agents and Amazon Nova 2.0 Lite

Build AI agents using the Strands framework with Amazon Nova 2.0 Lite. This notebook covers agent fundamentals, tools, and practical applications.

## Table of Contents

1. [**Installation and Setup**](#1.-Installation-and-Setup) — Environment configuration and API key authentication
2. [**Creating Your First Agent**](#2.-Creating-Your-First-Agent) — Basic agent creation with Hybrid Reasoning
3. [**Extending Agent Capabilities with Tools**](#3.-Extending-Agent-Capabilities-with-Tools) — System Tools, Community Tools, Custom Tools, and MCP
4. [**Multimodal Content Processing**](#4.-Multimodal-Content-Processing) — Working with images and documents
5. [**Streaming Responses**](#5.-Streaming-Responses) — Real-time response generation
6. [**Structured Output**](#6.-Structured-Output) — Type-safe responses with Pydantic models
7. [**Configuration and Best Practices**](#7.-Configuration-and-Best-Practices) — Model tuning and error handling
8. [**Practical Example: Research Assistant**](#8.-Practical-Example:-Research-Assistant) — Building a complete application

## What is an AI Agent?

An **AI agent** is a program that understands natural language, makes decisions, and uses tools to perform tasks—like searching the web, analyzing data, or interacting with external services.

## About Amazon Nova 2.0 Lite

**Amazon Nova 2.0 Lite** is a fast, cost-effective multimodal model that processes text, images, and documents. Key features include **Hybrid Reasoning** for improved accuracy and **System Tools** for built-in capabilities like web search and code execution.

## 1. Installation and Setup

First, let's install the required packages:

In [ ]:
# Install Strands Agents and community tools
!pip install strands-agents strands-agents-tools

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### AWS Setup Requirements

Before using Nova 2.0 Lite, you need:

1. **AWS Account** with Amazon Bedrock access
2. **IAM Permissions** - Appropriate permissions to invoke Bedrock models
3. **Amazon Bedrock API Key** - Generate an API key for authentication

#### Step 1: Verify Model Access

Access to Amazon Bedrock foundation models, including Amazon Nova 2.0 Lite, is enabled by default in all commercial AWS regions.

**To get started:**
1. Sign in to the [AWS Management Console](https://console.aws.amazon.com/)
2. Navigate to the [Amazon Bedrock console](https://console.aws.amazon.com/bedrock/)
3. Make sure you're in the **US East (N. Virginia)** region (us-east-1)
4. Select **Nova 2.0 Lite** from the Model Catalog and open it in the Playground, or invoke it directly via API

**IAM Permissions Required:**
- `bedrock:InvokeModel` — Required to call models
- `bedrock:InvokeTool` — Required for System Tools (web search, code interpreter)
- `aws-marketplace:Subscribe` — One-time setup for AWS Marketplace models

> **Note**: Once a model is enabled in your account, all users in the account can invoke it without needing Marketplace permissions.

#### Step 2: Generate an Amazon Bedrock API Key

Amazon Bedrock API keys provide a simple way to authenticate without complex credential configuration. For this tutorial, we'll create a **long-term API key** that expires in 30 days:

1. In the [Amazon Bedrock console](https://console.aws.amazon.com/bedrock/), select **API keys** from the left navigation pane
2. In the **Long-term API keys** tab, click **Generate long-term API keys**
3. Under **API key expiration**, select **30 days**
4. Click **Generate**
5. **Important**: Copy your API key immediately and store it securely - it will only be shown once!

> **Note**: Long-term API keys are recommended for exploration and development only. For production applications, use short-term API keys or IAM roles for enhanced security.

#### Step 3: Configure Your API Key

Once you have your API key, you'll set it as an environment variable that the Strands framework will automatically use for authentication:

In [ ]:
import os

# Set your Amazon Bedrock API key here
# Replace 'your-api-key-here' with the API key you generated from the Bedrock console
os.environ['AWS_BEARER_TOKEN_BEDROCK'] = 'your-api-key-here'

# The Strands framework uses boto3 under the hood, which automatically detects
# this environment variable and uses it for authentication with Amazon Bedrock.

print("Amazon Bedrock API key configured!")
print("Make sure you've replaced 'your-api-key-here' with your actual API key.")

In [ ]:
# Alternative: Set the API key in your terminal before running Jupyter
# This keeps your API key out of your notebook code

# For macOS/Linux, run this in your terminal:
# export AWS_BEARER_TOKEN_BEDROCK="your-api-key-here"

# For Windows (Command Prompt):
# setx AWS_BEARER_TOKEN_BEDROCK "your-api-key-here"

# For Windows (PowerShell):
# $env:AWS_BEARER_TOKEN_BEDROCK = "your-api-key-here"

# Then start Jupyter notebook from the same terminal session.

# Verify your API key is set (this will show if the environment variable exists)
import os
api_key = os.environ.get('AWS_BEARER_TOKEN_BEDROCK', '')
if api_key and api_key != 'your-api-key-here':
    print("API key is configured!")
else:
    print("Warning: API key not set or still using placeholder value.")
    print("Please set your API key in the cell above or via environment variable.")

## 2. Creating Your First Agent

Let's create a simple agent using Amazon Nova 2.0 Lite with **Hybrid Reasoning** enabled.

### What is Hybrid Reasoning?

Hybrid Reasoning is a powerful feature of Nova 2.0 Lite that allows the model to "think through" problems before providing a response. When enabled, the model generates internal reasoning steps that help it arrive at more accurate and well-considered answers.

To enable reasoning, you configure it using the `additional_request_fields` parameter in the `BedrockModel`. The reasoning configuration has two key settings:

- **type**: Set to `"enabled"` to turn on reasoning (default is `"disabled"`)
- **maxReasoningEffort**: Controls how deeply the model should think through problems. Options are:
  - `"low"` - Minimal reasoning, faster responses
  - `"medium"` - Balanced reasoning depth
  - `"high"` - Deep reasoning for complex problems

The `maxReasoningEffort` parameter sets a limit on how many tokens the model uses in its reasoning process. Always start with `"low"` and increase only if you need more accuracy for complex tasks.

In [ ]:
from strands import Agent
from strands.models import BedrockModel

# Create Nova 2.0 Lite model with Hybrid Reasoning enabled
nova_lite = BedrockModel(
    model_id="amazon.nova-2-lite-v1:0",  # Nova 2.0 Lite model ID
    temperature=0.0,  # Controls randomness (0.0 = deterministic, 1.0 = very random)
    region_name="us-east-1",  # Nova 2.0 Lite is available in us-east-1
    additional_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    }
)

# Create your first agent
agent = Agent(
    model=nova_lite,
    name="My First Agent",
    system_prompt="You are a helpful AI assistant. Be concise and friendly."
)

print("Agent created successfully with Hybrid Reasoning enabled!")

In [ ]:
# Have your first conversation
response = agent("Hello! Can you explain what you are in simple terms?")
print(response)

In [ ]:
# Ask a follow-up question
response = agent("What can you help me with?")
print(response)

## 3. Extending Agent Capabilities with Tools

Tools are functions that agents can call to perform specific tasks, extending capabilities beyond text generation.

### Types of Tools

| Tool Type | Description | Execution |
|-----------|-------------|-----------|
| **System Tools** | Built into Nova 2.0 Lite (`nova_grounding`, `nova_code_interpreter`) | Model executes internally |
| **Community Tools** | Pre-built tools from `strands-agents-tools` | Agent framework executes |
| **Custom Tools** | Your Python functions with `@tool` decorator | Agent framework executes |
| **MCP Tools** | External tools via Model Context Protocol servers | Agent framework executes |

**Key distinction:** System tools are unique to Nova 2.0—the model invokes and executes them internally during reasoning. All other tools follow the standard pattern where the agent framework handles execution based on the model's tool call requests.

### Configuration Parameters

When configuring `BedrockModel`, two parameters control different aspects:

- **`additional_request_fields`** — Configures model behavior (e.g., `reasoningConfig` for Hybrid Reasoning)
- **`additional_args`** — Configures API-level features (e.g., `toolConfig` for System Tools)

These are used together when you want both reasoning and system tools enabled.

### System Tools

System tools are model-managed capabilities that Nova 2.0 Lite can invoke directly during reasoning. No external code required—results are automatically incorporated into responses.

**Available system tools:**
- **`nova_grounding`** — Real-time web search with citations
- **`nova_code_interpreter`** — Python code execution

**Requirement:** IAM role needs `bedrock:InvokeTool` permission.

#### Web Search (Nova Grounding)

Enables real-time web search for current events and live data. The model formulates queries, executes searches, and incorporates results with citations.

```python
additional_args={
    "toolConfig": {
        "tools": [
            {
                "systemTool": {
                    "name": "nova_grounding"
                }
            }
        ]
    }
}
```

In [ ]:
# Create a model with web search (Nova Grounding) enabled
nova_with_web_search = BedrockModel(
    model_id="amazon.nova-2-lite-v1:0",
    temperature=0.7,
    region_name="us-east-1",
    max_tokens=10000,  # Set higher max tokens to accommodate web search results
    additional_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    },
    # Enable the nova_grounding system tool for web search
    additional_args={
        "toolConfig": {
            "tools": [
                {
                    "systemTool": {
                        "name": "nova_grounding"
                    }
                }
            ]
        }
    }
)

# Create an agent with web search capability
web_search_agent = Agent(
    model=nova_with_web_search,
    system_prompt="You are a helpful assistant with access to real-time web search. Use web search to find current information when needed."
)

print("Web search agent created!")

In [ ]:
# Test the web search agent with a question about current events
# The model will automatically search the web to find up-to-date information
response = web_search_agent("What are the top AI announcements from the past week?")
print(response)

#### Code Interpreter

Enables Python code execution for calculations, data analysis, and algorithmic tasks. The model writes code, executes it, and uses results (stdout/stderr) in its response. Errors are visible to the model for automatic debugging.

```python
additional_args={
    "toolConfig": {
        "tools": [
            {
                "systemTool": {
                    "name": "nova_code_interpreter"
                }
            }
        ]
    }
}
```

**Note:** Code execution may increase response time—consider higher timeout values.

In [ ]:
# Create a model with Code Interpreter enabled
nova_with_code_interpreter = BedrockModel(
    model_id="amazon.nova-2-lite-v1:0",
    temperature=0.7,
    region_name="us-east-1",
    max_tokens=10000,  # Set higher max tokens to accommodate code execution results
    additional_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    },
    # Enable the nova_code_interpreter system tool
    additional_args={
        "toolConfig": {
            "tools": [
                {
                    "systemTool": {
                        "name": "nova_code_interpreter"
                    }
                }
            ]
        }
    }
)

# Create an agent with code interpreter capability
code_interpreter_agent = Agent(
    model=nova_with_code_interpreter,
    system_prompt="You are a helpful assistant that can write and execute Python code to solve problems."
)

print("Code interpreter agent created!")

In [ ]:
# Test the code interpreter with a complex calculation
# The model will write and execute Python code to solve this problem
response = code_interpreter_agent(
    "Calculate how many seconds are in a leap year, and what percentage more that is compared to a regular year."
)
print(response)

### Community Tools

The `strands-agents-tools` package provides pre-built tools including calculators, time functions, HTTP requests, and more.

In [ ]:
# Import some useful tools from the community package
from strands_tools import calculator, current_time

# Create an agent with tools
agent_with_tools = Agent(
    model=nova_lite,
    tools=[calculator, current_time],
    system_prompt="You are a helpful assistant with access to a calculator and current time."
)

print("Agent with tools created!")

In [ ]:
# Test the calculator tool
response = agent_with_tools("What's 15 * 23 + 47?")
print(response)

In [ ]:
# Test the current time tool
response = agent_with_tools("What time is it right now?")
print(response)

### Creating Custom Tools

You can create your own tools using the `@tool` decorator:

In [ ]:
from strands import tool

@tool
def weather_info(city: str) -> str:
    """Get weather information for a city.
    
    Args:
        city: The name of the city
    """
    # This is a mock function - in reality, you'd call a weather API
    weather_data = {
        "new york": "Sunny, 72°F",
        "london": "Cloudy, 15°C",
        "tokyo": "Rainy, 18°C",
        "paris": "Partly cloudy, 20°C"
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        return f"Weather in {city}: {weather_data[city_lower]}"
    else:
        return f"Weather data not available for {city}. Try: New York, London, Tokyo, or Paris."

@tool
def text_analyzer(text: str) -> str:
    """Analyze text and provide statistics.
    
    Args:
        text: The text to analyze
    """
    words = text.split()
    chars = len(text)
    chars_no_spaces = len(text.replace(" ", ""))
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {len(words)}
- Characters: {chars}
- Characters (no spaces): {chars_no_spaces}
- Sentences: {sentences}
- Average word length: {chars_no_spaces / len(words):.1f} characters"""

print("Custom tools created!")

In [ ]:
# Create an agent with custom tools
custom_agent = Agent(
    model=nova_lite,
    tools=[weather_info, text_analyzer, calculator],
    system_prompt="You are a helpful assistant with weather, text analysis, and calculation capabilities."
)

# Test the weather tool
response = custom_agent("What's the weather like in Tokyo?")
print(response)

In [ ]:
# Test the text analyzer
response = custom_agent("Can you analyze this text: 'The quick brown fox jumps over the lazy dog. This sentence contains every letter of the alphabet!'")
print(response)

### MCP Tools

**Model Context Protocol (MCP)** connects agents to external tool servers for capabilities like documentation lookup, database access, or API integration.

**Prerequisites:**
- Install `uv` package manager: `pip install uv`
- Internet access to download MCP servers on first run
- The MCP server runs as a subprocess, so ensure your environment allows spawning processes

> **Note:** If the MCP server fails to start, check that `uvx` is available in your PATH and that you have network access. The first run may take longer as it downloads the server package.

In [ ]:
from mcp import stdio_client, StdioServerParameters
from strands.tools.mcp import MCPClient

# Create MCP client for AWS documentation
mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx",
        args=["awslabs.aws-documentation-mcp-server@latest"]
    )
))

# Use with context manager
with mcp_client:
    # Get tools from MCP server
    mcp_tools = mcp_client.list_tools_sync()
    
    # Create agent with MCP tools
    mcp_agent = Agent(
        model=nova_lite,
        tools=mcp_tools,
        system_prompt="You are an AWS expert assistant with access to AWS documentation."
    )
    
    # Ask about AWS services
    response = mcp_agent("What is AWS Lambda?")
    print(response)

## 4. Multimodal Content Processing

Nova 2.0 Lite can analyze images alongside text, making it ideal for tasks like architecture diagram review, document analysis, and visual content understanding.

**Why use multimodal input?** Instead of manually describing visual content, you can pass images directly to the model for analysis, extraction, or question-answering—saving time and improving accuracy for tasks like diagram interpretation, screenshot analysis, or visual QA.

### Image Input Format

Images are passed as content blocks using the Bedrock Converse API format. Supported formats include PNG, JPEG, GIF, and WebP, with a maximum payload size of 25 MB.

```python
{
    "image": {
        "format": "png",  # png, jpeg, gif, or webp
        "source": {
            "bytes": image_bytes
        }
    }
}
```

Let's demonstrate by analyzing an AWS architecture diagram:

In [ ]:
# Create a multimodal agent for architecture analysis
architecture_agent = Agent(
    model=nova_lite,
    system_prompt="""You are an AWS Solutions Architect assistant. 
Analyze architecture diagrams to identify components, data flows, and provide recommendations."""
)

# Load the AWS architecture diagram
with open("deployment_dashboard_architecture.png", "rb") as f:
    architecture_image = f.read()

# Analyze the architecture diagram
response = architecture_agent([
    {
        "image": {
            "format": "png",
            "source": {
                "bytes": architecture_image
            }
        }
    },
    {
        "text": """Analyze this AWS architecture diagram and provide:
1. A list of all AWS services shown
2. The data flow from user to backend
3. Key security components
4. One recommendation for improvement"""
    }
])

print(response)

## 5. Streaming Responses

**Why use streaming?** For longer responses, streaming displays output as it's generated rather than waiting for the complete response. This improves perceived latency and user experience, especially for complex queries that take several seconds to process.

Streaming is particularly useful for:
- Chat interfaces where users expect immediate feedback
- Long-form content generation (stories, reports, analysis)
- Applications where partial results are valuable

In [ ]:
import asyncio

async def stream_example():
    """Example of streaming responses from the agent."""
    print("Streaming response:")
    print("-" * 50)
    
    async for event in agent.stream_async("Tell me a short story about a robot learning to paint."):
        # Check if this is a text event
        if text_event := event.get("text_event"):
            print(text_event["text"], end="", flush=True)
    
    print("\n" + "-" * 50)
    print("Streaming complete!")

# Run the streaming example
await stream_example()

## 6. Structured Output

**Why use structured output?** Raw text responses require parsing and validation, which is error-prone. Structured output guarantees the model returns data in a predefined schema, making it reliable for programmatic use—like populating databases, generating API responses, or feeding data into downstream systems.

Use Pydantic models to define your expected output structure:

In [ ]:
from pydantic import BaseModel, Field

# Define output structure
class PersonInfo(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    occupation: str = Field(description="Occupation of the person")

# Get structured output
result = agent(
    "John Smith is a 30 year-old software engineer",
    structured_output_model=PersonInfo
)

person: PersonInfo = result.structured_output
print(f"Name: {person.name}")
print(f"Age: {person.age}")
print(f"Job: {person.occupation}")

## 7. Configuration and Best Practices

### Model Configuration

You can fine-tune the model's behavior using various configuration parameters. One of the most powerful features of Nova 2.0 models is **Hybrid Reasoning**.

### Understanding Hybrid Reasoning in Depth

Hybrid Reasoning enables the Nova 2.0 model to perform internal "thinking" before generating its final response. This is especially useful for tasks that require:

- **Complex problem-solving** (math, logic puzzles, multi-step reasoning)
- **Careful analysis** (comparing options, weighing trade-offs)
- **Accuracy-critical responses** (factual questions, technical explanations)

#### How Reasoning Works

When reasoning is enabled, the model's response includes two parts:
1. **Reasoning Content**: The model's internal thought process (can be accessed separately)
2. **Text Response**: The final answer presented to the user

#### The `reasoningConfig` Parameter

You enable reasoning through the `additional_request_fields` parameter with a `reasoningConfig` object:

```python
additional_request_fields={
    "reasoningConfig": {
        "type": "enabled",      # or "disabled" (default)
        "maxReasoningEffort": "low"  # Required when type is "enabled"
    }
}
```

#### Understanding `maxReasoningEffort` (Thinking Budget)

The `maxReasoningEffort` parameter controls the **reasoning budget** — how many tokens the model allocates to its thinking process. This directly impacts:

| Effort Level | Token Budget | Best For | Trade-offs |
|--------------|--------------|----------|------------|
| `"low"` | Minimal tokens | Simple tasks, quick responses | Fastest, but may miss nuances |
| `"medium"` | Moderate tokens | Balanced tasks, most use cases | Good balance of speed and accuracy |
| `"high"` | Maximum tokens | Complex problems, critical accuracy | Slowest, highest token usage |

#### Best Practices for Reasoning Effort

1. **Start with `"low"`**: Always begin with low effort and increase only if needed
2. **Use `"medium"` for most tasks**: Provides a good balance for general-purpose applications
3. **Reserve `"high"` for complex tasks**: Math problems, multi-step analysis, or when accuracy is critical
4. **Consider latency**: Higher effort means longer response times
5. **Monitor token usage**: Reasoning tokens count toward your usage, so balance cost vs. accuracy

In [ ]:
# Create a more configured model with medium reasoning effort
configured_nova = BedrockModel(
    model_id="amazon.nova-2-lite-v1:0",
    temperature=0.3,  # Lower temperature for more focused responses
    top_p=0.8,        # Controls diversity of responses
    max_tokens=1000,  # Maximum response length
    stop_sequences=["END", "STOP"],  # Sequences that stop generation
    region_name="us-east-1",
    additional_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "medium"  # Balanced reasoning for most tasks
        }
    }
)

# Create agent with configured model
precise_agent = Agent(
    model=configured_nova,
    system_prompt="You are a precise, factual assistant. Keep responses concise and accurate."
)

response = precise_agent("Explain quantum computing in exactly 3 sentences.")
print(response)

### Error Handling

When working with Bedrock models, you may encounter these common errors:

| Error | Cause | Solution |
|-------|-------|----------|
| `AccessDeniedException` | Missing IAM permissions | Add `bedrock:InvokeModel` to your IAM policy |
| `ValidationException` | Invalid model ID or parameters | Verify model ID and region |
| `ResourceNotFoundException` | Model not available in region | Check model availability in your region |
| `ThrottlingException` | Rate limit exceeded | Implement exponential backoff retry |
| `ReadTimeoutError` | Response took too long | Increase timeout, especially for System Tools |

Here's a basic error handling pattern:

In [ ]:
def safe_agent_call(agent, message):
    """Safely call an agent with error handling."""
    try:
        response = agent(message)
        return response
    except Exception as e:
        return f"Error occurred: {str(e)}. Please check your AWS credentials and model access."

# Test error handling
response = safe_agent_call(agent, "Hello, how are you?")
print(response)

## 8. Practical Example: Research Assistant

Let's build a research assistant that combines Nova 2.0 Lite's **Hybrid Reasoning** with **Nova Grounding** (web search) to gather real-time information, analyze it, and provide well-sourced answers.

This example demonstrates:
- System tools for live web search with citations
- Hybrid Reasoning for synthesizing information from multiple sources
- Structured system prompts for consistent output formatting

In [ ]:
# Create a Nova model with web search (Nova Grounding) and reasoning enabled
research_model = BedrockModel(
    model_id="amazon.nova-2-lite-v1:0",
    temperature=0.3,  # Lower temperature for factual research
    region_name="us-east-1",
    max_tokens=10000,
    additional_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "medium"  # Medium effort for thorough research
        }
    },
    additional_args={
        "toolConfig": {
            "tools": [
                {
                    "systemTool": {
                        "name": "nova_grounding"
                    }
                }
            ]
        }
    }
)

# Create the research assistant agent
research_assistant = Agent(
    model=research_model,
    system_prompt="""You are a research assistant with access to real-time web search.

When researching a topic:
1. Search for current, authoritative sources
2. Cross-reference information from multiple sources when possible
3. Cite your sources with URLs
4. Distinguish between facts and opinions
5. Note if information might be outdated or uncertain

Format your responses with clear sections:
- **Summary**: Brief overview of findings
- **Key Points**: Main facts discovered
- **Sources**: List of references used

Be thorough but concise. Always prioritize accuracy over speed."""
)

print("Research assistant created with web search and reasoning capabilities!")

In [ ]:
# Test the research assistant with a current events question
# The agent will search the web for real-time information
response = research_assistant(
    "What are the latest developments in quantum computing? Focus on recent breakthroughs and their practical applications."
)
print(response)

## Summary

You've learned to build AI agents with Strands and Amazon Nova 2.0 Lite, including:

- **Setup**: API key authentication and model configuration
- **Hybrid Reasoning**: Enabling the model to "think through" problems with configurable effort levels
- **Tools**: System Tools (web search, code interpreter), Community Tools, Custom Tools, and MCP integration
- **Content Processing**: Multimodal inputs, streaming responses, and structured outputs

### Resources
- [Strands Documentation](https://strandsagents.com)
- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Amazon Nova Documentation](https://docs.aws.amazon.com/nova/)

### Next Steps
1. Experiment with different tools and reasoning configurations
2. Build an application for your use case
3. Explore multi-agent patterns for complex workflows